<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/agents_and_routing/examples_agents_with_AtomicAgents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="llama3.2"
emb_modelid="mxbai-embed-large"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {emb_modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
from google.colab import userdata

ngrok.kill()
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok_tunnel = ngrok.connect(addr="11434", proto="http", bind_tls=True, host_header="localhost:11434" )
print("OLLama URL:", ngrok_tunnel.public_url)

llama_url = ngrok_tunnel.public_url

In [2]:
!pip install atomic-agents instructor --quiet

In [3]:
import os
from atomic_agents.lib.components.system_prompt_generator import SystemPromptGenerator

system_prompt_generator = SystemPromptGenerator(
    background=[
        'This assistant is a general-purpose AI designed to be helpful and friendly.',
    ],
    steps=[
        'Understand the user\'s input and provide a relevant response.',
        'Respond to the user.'
    ],
    output_instructions=[
        'Provide helpful and relevant information to assist the user.',
        'Be friendly and respectful in all interactions.',
        'Always answer in rhyming verse.'
    ]
)

system_prompt_generator.generate_prompt()

"# IDENTITY and PURPOSE\n- This assistant is a general-purpose AI designed to be helpful and friendly.\n\n# INTERNAL ASSISTANT STEPS\n- Understand the user's input and provide a relevant response.\n- Respond to the user.\n\n# OUTPUT INSTRUCTIONS\n- Provide helpful and relevant information to assist the user.\n- Be friendly and respectful in all interactions.\n- Always answer in rhyming verse.\n- Always respond using the proper JSON schema.\n- Always use the available additional information and context to enhance the response."

In [11]:
import instructor
import openai
from openai import OpenAI
from atomic_agents.agents.base_agent import BaseAgent, BaseAgentConfig
from atomic_agents.lib.components.agent_memory import AgentMemory




llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )

client = instructor.from_openai(
    llm,
    mode=instructor.Mode.JSON,
)

memory = AgentMemory()

agent = BaseAgent(
    config=BaseAgentConfig(
        client=client,
        system_prompt_generator=system_prompt_generator,
        model=modelid,
        memory=memory,
    )
)

In [12]:
import json
from atomic_agents.lib.base.base_io_schema import BaseAgentResponseSchema

print(json.dumps(BaseAgentResponseSchema.model_json_schema(), indent=2))

ModuleNotFoundError: No module named 'atomic_agents.lib.schemas'

In [ ]:
while True:
    user_input = input('You: ')
    if user_input.lower() in ['/exit', '/quit']:
        print('Exiting chat...')
        break

    response = agent.run(agent.input_schema(chat_message=user_input))
    print(f'Agent: {response.chat_message}')